# Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
from datetime import datetime
import os

2024-01-29 16:07:57.753386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Seed for reproducing

In [2]:
np.random.seed(888)
tf.random.set_seed(404)

# Constants

In [3]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

In [4]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)


# Data Preprocessing

In [5]:
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_test = np.eye(NR_CLASSES)[y_test]

# Create validation dataset from training data

In [6]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Neural Network Architecture

In [7]:
nr_epochs = 50
learning_rate = 1e-3
n_hidden1 = 512
n_hidden2 = 64

# Define the Model

In [8]:
model = Sequential([
    Flatten(input_shape=(TOTAL_INPUTS,)),
    Dense(n_hidden1, activation='relu'),
    Dropout(0.8),
    Dense(n_hidden2, activation='relu'),
    Dense(NR_CLASSES, activation='softmax')
])

# Compile the Model

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# TensorBoard Setup

In [10]:
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Training the Model with TensorBoard

In [11]:
history = model.fit(
    x_train, y_train, 
    epochs=nr_epochs, 
    batch_size=1000, 
    validation_data=(x_val, y_val), 
    callbacks=[tensorboard_callback]
)

Epoch 1/50
50/50 [==============================] - 2s 23ms/step - loss: 0.9954 - accuracy: 0.6813 - val_loss: 0.3377 - val_accuracy: 0.9047
Epoch 2/50
50/50 [==============================] - 1s 16ms/step - loss: 0.4118 - accuracy: 0.8761 - val_loss: 0.2365 - val_accuracy: 0.9312
Epoch 3/50
50/50 [==============================] - 1s 16ms/step - loss: 0.3238 - accuracy: 0.9017 - val_loss: 0.1948 - val_accuracy: 0.9420
Epoch 4/50
50/50 [==============================] - 1s 17ms/step - loss: 0.2769 - accuracy: 0.9181 - val_loss: 0.1676 - val_accuracy: 0.9498
Epoch 5/50
50/50 [==============================] - 1s 17ms/step - loss: 0.2456 - accuracy: 0.9267 - val_loss: 0.1544 - val_accuracy: 0.9535
Epoch 6/50
50/50 [==============================] - 1s 17ms/step - loss: 0.2247 - accuracy: 0.9332 - val_loss: 0.1384 - val_accuracy: 0.9593
Epoch 7/50
50/50 [==============================] - 1s 18ms/step - loss: 0.2056 - accuracy: 0.9394 - val_loss: 0.1325 - val_accuracy: 0.9601
Epoch 8/50
50

# Saving the Model

In [12]:
model.save('SavedModel/my_model')

2024-01-29 16:08:57.556618: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2024-01-29 16:08:57.773605: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: SavedModel/my_model/assets


INFO:tensorflow:Assets written to: SavedModel/my_model/assets


# Making a Prediction

In [13]:
def make_prediction(image_path):
    img = Image.open(image_path)
    bw = img.convert('L')
    img_array = np.invert(bw)
    test_img = img_array.ravel() / 255.0
    prediction = model.predict(np.array([test_img]))
    return np.argmax(prediction)

# Example of making a prediction

In [14]:
prediction = make_prediction('MNIST/test_img.png')
print(f'Prediction for test image is {prediction}')

1/1 [==============================] - 0s 87ms/step
Prediction for test image is 2


# Testing and Evaluation

In [15]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy:.2%}")

313/313 [==============================] - 1s 2ms/step - loss: 0.0694 - accuracy: 0.9802
Test loss: 0.06939810514450073, Test accuracy: 98.02%
